In [1]:
import csv
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('raw_data.db')
cur = conn.cursor()
        
for year in ["2012", "2016", "2020", "2023"]:
    cur.execute(f'''CREATE TABLE IF NOT EXISTS tab{year} (
                iteration_number INTEGER,
                party_number INTEGER,
                samples INTEGER,
                diff INTEGER
                );''')

    # Read and insert data from CSV
    with open(f'./raw_data/1m-{year}.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header if exists
        for row in reader:
            cur.execute(f'''INSERT INTO tab{year} (iteration_number, party_number, samples, diff)
                            VALUES (?, ?, ?, ?);''', row)
        
# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
# 2023
with sqlite3.connect('first_change.db') as conn:
    cur = conn.cursor()
    cur.execute('''DELETE FROM tab2023 WHERE party_number IN (1, 2, 4, 6, 7, 8, 9, 10, 11, 13, 14, 19, 20, 21, 22, 24);''')
    conn.commit()
# 2020
with sqlite3.connect('first_change.db') as conn:
    cur = conn.cursor()
    cur.execute('''DELETE FROM tab2020 WHERE party_number IN (1, 5, 7, 8, 10, 13, 14, 16, 17, 18, 20, 21, 23, 25);''')
    conn.commit()
# 2016
with sqlite3.connect('first_change.db') as conn:
    cur = conn.cursor()
    cur.execute('''DELETE FROM tab2016 WHERE party_number IN (1, 2, 4, 5, 7, 8, 9, 10, 13, 14, 15, 18, 22);''')
    conn.commit()
# 2012
with sqlite3.connect('first_change.db') as conn:
    cur = conn.cursor()
    cur.execute('''DELETE FROM tab2012 WHERE party_number IN (1, 3, 7, 8, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 23, 25, 26);''')
    conn.commit()

    

In [ ]:
# Connect to SQLite database
try:
    with sqlite3.connect('raw_data.db') as conn:
        cur = conn.cursor()

        # Delete records where party_number is 3
        cur.execute('''DELETE FROM tab1m WHERE party_number IN (1, 2, 4, 6, 7, 8, 9, 10, 11, 13, 14, 19, 20, 21, 22, 24);''')

        # Commit changes
        conn.commit()
except sqlite3.Error as e:
    print("SQLite error:", e)
except Exception as e:
    print("Unexpected error:", e)
for year in ["2023", "2020", "2016", "2012"]:
    with sqlite3.connect('first_change.db') as conn:
            cur = conn.cursor()

            cur.execute(f'''
                SELECT iteration_number, party_number, MIN(samples) 
                FROM tab{year} 
                WHERE diff > 0 
                GROUP BY iteration_number, party_number;
            ''')

            results = cur.fetchall()

            with open(f'lowest_samples{year}.csv', 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(['iteration_number', 'party_number', 'lowest_samples'])
                csvwriter.writerows(results)



In [7]:
with sqlite3.connect('raw_data.db') as conn:
        cur = conn.cursor()

        # Select the lowest samples for each combination of iteration_number and party_number
        cur.execute('''
            SELECT iteration_number, party_number, MIN(samples) 
            FROM tab1m 
            WHERE diff > 0 
            GROUP BY iteration_number, party_number;
        ''')

        # Fetch the results
        results = cur.fetchall()

        # Write results to a CSV file
        with open('lowest_samples.csv', 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['iteration_number', 'party_number', 'lowest_samples'])  # Write header
            csvwriter.writerows(results)

        print("Results exported to lowest_samples.csv")
year = 2020
subjects = set(x if x not in (1, 5, 7, 8, 10, 13, 14, 16, 17, 18, 20, 21, 23, 25) else 12 for x in range(1, 26))


for subject in subjects:
    with sqlite3.connect('first_change.db') as conn:
            cur = conn.cursor()

            cur.execute(f'''
                SELECT iteration_number, party_number, MIN(samples) 
                FROM tab{year} 
                WHERE diff > 0 AND party_number = {subject}
                GROUP BY iteration_number, party_number;
            ''')

            results = cur.fetchall()

            with open(f'lowest_samples{year}-p{subject}.csv', 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(['iteration_number', 'party_number', 'lowest_samples'])
                csvwriter.writerows(results)


Results exported to lowest_samples.csv


In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem, t

# Function to calculate confidence interval
def confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    return m, m - h, m + h

# Read data from CSV
df = pd.read_csv('lowest_samples.csv')

# Calculate mean and confidence intervals
group_means = df.groupby(['iteration_number', 'party_number']).agg({'lowest_samples': 'mean'}).reset_index()
group_means['mean'], group_means['lower_ci'], group_means['upper_ci'] = zip(*group_means['lowest_samples'].apply(confidence_interval))

# Plotting
plt.figure(figsize=(10, 6))
sns.lineplot(data=group_means, x='iteration_number', y='mean', hue='party_number', marker='o', ci=None)
plt.fill_between(group_means['iteration_number'], group_means['lower_ci'], group_means['upper_ci'], alpha=0.3)
plt.title('Mean and Confidence Intervals of Lowest Samples')
plt.xlabel('Iteration Number')
plt.ylabel('Lowest Samples')
plt.legend(title='Party Number')
plt.show()
#  for x in range(1, 26)]



# with sqlite3.connect('first_change.db') as conn:
#         cur = conn.cursor()

#         cur.execute(f'''
#             SELECT iteration_number, party_number, MIN(samples) 
#             FROM tab{year} 
#             WHERE diff > 0 AND party_number = 12
#             GROUP BY iteration_number, party_number;
#         ''')

#         results = cur.fetchall()

#         with open(f'lowest_samples{year}-p{subject}.csv', 'w', newline='') as csvfile:
#             csvwriter = csv.writer(csvfile)
#             csvwriter.writerow(['iteration_number', 'party_number', 'lowest_samples'])
#             csvwriter.writerows(results)


TypeError: object of type 'float' has no len()

In [5]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np
# from scipy.stats import sem, t

# # Function to calculate confidence interval
# def confidence_interval(data, confidence=0.95):
#     n = len(data)
#     m = np.mean(data)
#     std_err = sem(data)
#     h = std_err * t.ppf((1 + confidence) / 2, n - 1)
#     return m, m - h, m + h

# # Read data from CSV
# df = pd.read_csv('lowest_samples.csv')

# # Calculate mean and confidence intervals
# group_means = df.groupby(['iteration_number', 'party_number']).agg({'lowest_samples': 'mean'}).reset_index()
# group_means['mean'], group_means['lower_ci'], group_means['upper_ci'] = zip(*group_means['lowest_samples'].apply(confidence_interval))

# # Plotting
# plt.figure(figsize=(10, 6))
# sns.lineplot(data=group_means, x='iteration_number', y='mean', hue='party_number', marker='o', ci=None)
# plt.fill_between(group_means['iteration_number'], group_means['lower_ci'], group_means['upper_ci'], alpha=0.3)
# plt.title('Mean and Confidence Intervals of Lowest Samples')
# plt.xlabel('Iteration Number')
# plt.ylabel('Lowest Samples')
# plt.legend(title='Party Number')
# plt.show()
